In [ ]:
import numpy as np
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image
import matplotlib.pyplot as plt
import os
import numpy
from PIL import Image
import torch

if not os.path.exists('./clouds_remove'):
    os.mkdir('./clouds_remove')

def min_max_normalization(tensor, min_value, max_value):
    min_tensor = tensor.min()
    tensor = (tensor - min_tensor)
    max_tensor = tensor.max()
    tensor = tensor / max_tensor
    tensor = tensor * (max_value - min_value) + min_value
    return tensor


img_transform = transforms.Compose([
    transforms.Lambda(lambda tensor:min_max_normalization(tensor, 0, 1)),
])
        

class LandsatDataset():

    def __init__(self, root_dir, transform=None):

        self.root_dir = root_dir
        self.transform = transform
#         self.batch_size=batch_size
    def __len__(self):
        return len(os.listdir(self.root_dir))

    def __getitem__(self, idx):
#         samples=[]
#         for i in range(self.batch_size):
        img_name = os.listdir(self.root_dir)[idx]#[idx*self.batch_size+i]
        img = Image.open(self.root_dir+img_name)
        image = numpy.array(img)[4000:4700,3000:3500]
        image_tensor=torch.from_numpy(image)
        image_tensor=image_tensor.cuda()
        #samples.append(image_tensor)
        #tensor=torch.cat((*samples),(0))
#         if self.transform:
#             sample = self.transform(sample)
        return image_tensor

num_epochs=10000
learning_rate = 4*1e-4
batch_size=55

dataset = LandsatDataset('./landsat8_142_49_truecolor_copy/',transform=None)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(55, 55),
            nn.ReLU(True))
        self.decoder = nn.Sequential(
            nn.Linear(55, 55),
            nn.ReLU(True),
            nn.Sigmoid())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x



model = autoencoder().cuda()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(
    model.parameters(), lr=learning_rate, weight_decay=1e-5)

for data in dataloader:
    
    img_B = min_max_normalization(data[0:,0:,0:,0].float(),0.0,1.0)
    img_G = min_max_normalization(data[0:,0:,0:,1].float(),0.0,1.0)
    img_R = min_max_normalization(data[0:,0:,0:,2].float(),0.0,1.0)
    img_B=img_B.view(img_B.size(0),-1).float()
    img_B = torch.transpose(img_B, 0, 1)
    img_G=img_G.view(img_G.size(0),-1).float()
    img_G = torch.transpose(img_G, 0, 1)
    img_R=img_R.view(img_R.size(0),-1).float()
    img_R = torch.transpose(img_R, 0, 1)
    for epoch in range(num_epochs):
        img_B = Variable(img_B).cuda()
        output_B = model(img_B)
        output_G = model(img_G)
        output_R = model(img_R)
        loss = criterion(output_B, img_B)
        MSE_loss = nn.MSELoss()(output_B, img_B)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
        if (epoch+1)%1000==0:
            print('epoch [{}/{}], loss:{:.4f}, MSE_loss:{:.4f}'
              .format(epoch + 1, num_epochs, loss.data[0], MSE_loss.data[0]))
            
    
#     img_fb= torch.transpose(img_B,0,1)
#     img_fg= torch.transpose(img_G,0,1)
#     img_fr= torch.transpose(img_R,0,1)
    output_fb=torch.transpose(output_B,0,1).view(data.size(0),1,data.size(1),data.size(2))
    output_fg=torch.transpose(output_G,0,1).view(data.size(0),1,data.size(1),data.size(2))
    output_fr=torch.transpose(output_R,0,1).view(data.size(0),1,data.size(1),data.size(2))
    output=torch.cat((output_fb,output_fg,output_fr),1).cpu().data
    # MIDIMG= model.encoder(img).view(img.size(0),1,56,56).cpu().data
    # save_image(x, './mlp_pansharpened/x_{}.png'.format(epoch))
    save_image(output, './clouds_remove/x_output.png')   
        
    input_fb=torch.transpose(img_B,0,1).view(data.size(0),1,data.size(1),data.size(2))
    input_fg=torch.transpose(img_G,0,1).view(data.size(0),1,data.size(1),data.size(2))
    input_fr=torch.transpose(img_R,0,1).view(data.size(0),1,data.size(1),data.size(2))
    ini=torch.cat((input_fb,input_fg,input_fr),1).cpu().data
    save_image(ini, './clouds_remove/x_inputlayer.png')
    # save_image(MIDIMG, './mlp_pansharpened/x_midlayer{}.png'.format(epoch))

    


In [25]:
import numpy as np
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image
import matplotlib.pyplot as plt
import os
import numpy
from PIL import Image
import torch
from osgeo import gdal


def min_max_normalization(tensor, min_value, max_value):
    min_tensor = tensor.min()
    tensor = (tensor - min_tensor)
    max_tensor = tensor.max()
    tensor = tensor / max_tensor
    tensor = tensor * (max_value - min_value) + min_value
    return tensor

class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(7, 3),
            nn.ReLU(True))
        self.decoder = nn.Sequential(
            nn.Linear(3, 7),
            nn.ReLU(True),
            nn.Sigmoid())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x




os.chdir('E:/pyScripts/pytorch/data/142_49/LC08_L1TP_142049_20170120_20170311_01_T1/')

band_fn='LC08_L1TP_142049_20170120_20170311_01_T1_B1.TIF'
in_ds = gdal.Open(band_fn)
in_band=in_ds.GetRasterBand(1)
band1=torch.from_numpy(in_band.ReadAsArray().astype('float'))
band1=min_max_normalization(band1,0.0,1.0)


band_fn='LC08_L1TP_142049_20170120_20170311_01_T1_B2.TIF'
in_ds = gdal.Open(band_fn)
in_band=in_ds.GetRasterBand(1)
band2=torch.from_numpy(in_band.ReadAsArray().astype('float'))
band2=min_max_normalization(band2,0.0,1.0)


band_fn='LC08_L1TP_142049_20170120_20170311_01_T1_B3.TIF'
in_ds = gdal.Open(band_fn)
in_band=in_ds.GetRasterBand(1)
band3=torch.from_numpy(in_band.ReadAsArray().astype('float'))
band3=min_max_normalization(band3,0.0,1.0)


band_fn='LC08_L1TP_142049_20170120_20170311_01_T1_B4.TIF'
in_ds = gdal.Open(band_fn)
in_band=in_ds.GetRasterBand(1)
band4=torch.from_numpy(in_band.ReadAsArray().astype('float'))
band4=min_max_normalization(band4,0.0,1.0)


band_fn='LC08_L1TP_142049_20170120_20170311_01_T1_B5.TIF'
in_ds = gdal.Open(band_fn)
in_band=in_ds.GetRasterBand(1)
band5=torch.from_numpy(in_band.ReadAsArray().astype('float'))
band5=min_max_normalization(band5,0.0,1.0)


band_fn='LC08_L1TP_142049_20170120_20170311_01_T1_B6.TIF'
in_ds = gdal.Open(band_fn)
in_band=in_ds.GetRasterBand(1)
band6=torch.from_numpy(in_band.ReadAsArray().astype('float'))
band6=min_max_normalization(band6,0.0,1.0)


band_fn='LC08_L1TP_142049_20170120_20170311_01_T1_B7.TIF'
in_ds = gdal.Open(band_fn)
in_band=in_ds.GetRasterBand(1)
band7=torch.from_numpy(in_band.ReadAsArray().astype('float'))
band7=min_max_normalization(band7,0.0,1.0)


num_epochs=1
learning_rate = 4*1e-4
batch_size=55

model = autoencoder().cuda()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(
    model.parameters(), lr=learning_rate, weight_decay=1e-5)

for epoch in range(num_epochs):
    for i in range(0,band1.size(0)):
        b1_input=band1[i].view(-1,1)
        b2_input=band2[i].view(-1,1)
        b3_input=band3[i].view(-1,1)
        b4_input=band4[i].view(-1,1)
        b5_input=band5[i].view(-1,1)
        b6_input=band6[i].view(-1,1)
        b7_input=band7[i].view(-1,1)
        bands_input = torch.cat((
            b1_input,b2_input,b3_input,b4_input,b5_input,b6_input,b7_input),1)
        bands_input=Variable(bands_input).cuda()
        output=model(bands_input)
        loss = criterion(output, bands_input)
        MSE_loss = nn.MSELoss()(output, bands_input)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # ===================log========================
    if (epoch+1)%1==0:
        print('epoch [{}/{}], loss:{:.4f}, MSE_loss:{:.4f}'
          .format(epoch + 1, num_epochs, loss.data[0], MSE_loss.data[0]))
        
        seven_total_inputs=torch.cat(
            (band1.view(-1,1),band2.view(-1,1),band3.view(-1,1),band4.view(-1,1),band5.view(-1,1),band6.view(-1,1),band7.view(-1,1)),1)
        print(seven_total_inputs.size())
        seven_total_outputs=model(seven_total_inputs)
        print(seven_total_outputs.size())
        
        gtiff_driver = gdal.GetDriverByName('GTiff')
        out_ds = gtiff_driver.Create(
            'out_seven_bands.tif',in_band.XSize, in_band.YSize, 7, in_band.DataType)
        out_ds.SetProjection(in_ds.GetProjection())
        out_ds.SetGeoTransform(in_ds.GetGeoTransform())
        for j in range(7):
            out_band = out_ds.GetRasterBand(j)
            out_band.WriteArray(
                seven_total_outputs[:][j].view(band1.size(0),band1.size(1)))
        out_ds.FlushCache()
        for k in range(1, 7):
            out_ds.GetRasterBand(k).ComputeStatistics(False)
        del out_ds

RuntimeError: CUDA error: out of memory

In [28]:

print(band1[0])

tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0', dtype=torch.float64)


In [2]:
import numpy as np
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image
import matplotlib.pyplot as plt
import os
import numpy
from PIL import Image
import torch
from osgeo import gdal

os.chdir('E:/pyScripts/pytorch/data/142_49/LC08_L1TP_142049_20170120_20170311_01_T1/')
band_fn='LC08_L1TP_142049_20170120_20170311_01_T1_B1.TIF'
in_ds = gdal.Open(band_fn)
in_band=in_ds.GetRasterBand(1)
band1=in_band.ReadAsArray()

print(in_band.XSize, in_band.YSize, 7, in_band.DataType)
print(in_ds.GetProjection())
print(in_ds.GetGeoTransform())


7591 7751 7 2
PROJCS["WGS 84 / UTM zone 44N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",81],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32644"]]
(386385.0, 30.0, 0.0, 1873815.0, 0.0, -30.0)


In [8]:
import numpy as np
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image
import matplotlib.pyplot as plt
import os
import numpy
from PIL import Image
import torch
from osgeo import gdal


def min_max_normalization(tensor, min_value, max_value):
    min_tensor = tensor.min()
    tensor = (tensor - min_tensor)
    max_tensor = tensor.max()
    tensor = tensor / max_tensor
    tensor = tensor * (max_value - min_value) + min_value
    return tensor

class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(7, 3),
            nn.ReLU(True)
        )
        self.decoder = nn.Sequential(
            nn.Linear(3, 7),
            nn.ReLU(True)
            #nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x




os.chdir('E:/pyScripts/pytorch/data/142_49/LC08_L1TP_142049_20170120_20170311_01_T1/')

band_fn='LC08_L1TP_142049_20170120_20170311_01_T1_B1.TIF'
in_ds = gdal.Open(band_fn)
in_band=in_ds.GetRasterBand(1)
band1=torch.from_numpy(in_band.ReadAsArray().astype('float'))
band1=min_max_normalization(band1,0.0,1.0)


band_fn='LC08_L1TP_142049_20170120_20170311_01_T1_B2.TIF'
in_ds = gdal.Open(band_fn)
in_band=in_ds.GetRasterBand(1)
band2=torch.from_numpy(in_band.ReadAsArray().astype('float'))
band2=min_max_normalization(band2,0.0,1.0)


band_fn='LC08_L1TP_142049_20170120_20170311_01_T1_B3.TIF'
in_ds = gdal.Open(band_fn)
in_band=in_ds.GetRasterBand(1)
band3=torch.from_numpy(in_band.ReadAsArray().astype('float'))
band3=min_max_normalization(band3,0.0,1.0)


band_fn='LC08_L1TP_142049_20170120_20170311_01_T1_B4.TIF'
in_ds = gdal.Open(band_fn)
in_band=in_ds.GetRasterBand(1)
band4=torch.from_numpy(in_band.ReadAsArray().astype('float'))
band4=min_max_normalization(band4,0.0,1.0)


band_fn='LC08_L1TP_142049_20170120_20170311_01_T1_B5.TIF'
in_ds = gdal.Open(band_fn)
in_band=in_ds.GetRasterBand(1)
band5=torch.from_numpy(in_band.ReadAsArray().astype('float'))
band5=min_max_normalization(band5,0.0,1.0)


band_fn='LC08_L1TP_142049_20170120_20170311_01_T1_B6.TIF'
in_ds = gdal.Open(band_fn)
in_band=in_ds.GetRasterBand(1)
band6=torch.from_numpy(in_band.ReadAsArray().astype('float'))
band6=min_max_normalization(band6,0.0,1.0)


band_fn='LC08_L1TP_142049_20170120_20170311_01_T1_B7.TIF'
in_ds = gdal.Open(band_fn)
in_band=in_ds.GetRasterBand(1)
band7=torch.from_numpy(in_band.ReadAsArray().astype('float'))
band7=min_max_normalization(band7,0.0,1.0)


num_epochs=10


#batch_size=55

model = autoencoder().cuda()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(
    model.parameters(), lr=learning_rate, weight_decay=1e-5)


learning_rate = 1e-4
for epoch in range(num_epochs):
    atuple=[]
    for i in range(0,band1.size(0)):
        b1_input=band1[i].view(-1,1)
        b2_input=band2[i].view(-1,1)
        b3_input=band3[i].view(-1,1)
        b4_input=band4[i].view(-1,1)
        b5_input=band5[i].view(-1,1)
        b6_input=band6[i].view(-1,1)
        b7_input=band7[i].view(-1,1)
        bands_input = torch.cat((
            b1_input,b2_input,b3_input,b4_input,b5_input,b6_input,b7_input),1).float()
        bands_input=bands_input.cuda()
        output=model(bands_input)
        loss = criterion(output, bands_input)
        MSE_loss = nn.MSELoss()(output, bands_input)
        # ===================backward====================
        optimizer.zero_grad()
#         MSE_loss.backward()
        loss.backward()
        optimizer.step()
        atuple.append(output)
        # ===================log========================
    if (epoch+1)%1==0:
        print('epoch [{}/{}], loss:{:.4f}, MSE_loss:{:.4f}'.format(epoch + 1, num_epochs, loss.data[0], MSE_loss.data[0]))
    
atuple=tuple(atuple)
#print(len(atuple))
inputdata = torch.cat(atuple,0)
#print(inputdata.size())

gtiff_driver = gdal.GetDriverByName('GTiff')
out_ds = gtiff_driver.Create(
    'out_seven_bands.tif',band1.size(1), band1.size(0), 7, in_band.DataType)
out_ds.SetProjection(in_ds.GetProjection())
# out_ds.SetGeoTransform((386385.0+30*3000, 30.0, 0.0, 1873815.0-30.0*3751, 0.0, -30.0),)
out_ds.SetGeoTransform(in_ds.GetGeoTransform())
for j in range(7):
    out_band = out_ds.GetRasterBand((7-j))
    readyData=(inputdata[:,j].view(band1.size(0),band1.size(1))).cpu().detach().numpy()
    out_band.WriteArray(readyData)
out_ds.FlushCache()
for k in range(1, 7):
    out_ds.GetRasterBand(k).ComputeStatistics(False)
del out_ds
readyData_blue=(inputdata[:,2].view(1,band1.size(0),band1.size(1))).cpu()
readyData_green=(inputdata[:,3].view(1,band1.size(0),band1.size(1))).cpu()
readyData_red=(inputdata[:,4].view(1,band1.size(0),band1.size(1))).cpu()
readyData_BGR=torch.cat((readyData_blue,readyData_green,readyData_red),0)
save_image(readyData_BGR,'out_BRG.png')
print('all set!')

RuntimeError: $ Torch: not enough memory: you tried to allocate 0GB. Buy new RAM! at ..\aten\src\TH\THGeneral.cpp:204

In [4]:

model = autoencoder().cuda()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(
    model.parameters(), lr=learning_rate, weight_decay=1e-5)

learning_rate = 1e-4
for epoch in range(num_epochs):
    atuple=[]
    for i in range(0,band1.size(0)):
        b1_input=band1[i].view(-1,1)
        b2_input=band2[i].view(-1,1)
        b3_input=band3[i].view(-1,1)
        b4_input=band4[i].view(-1,1)
        b5_input=band5[i].view(-1,1)
        b6_input=band6[i].view(-1,1)
        b7_input=band7[i].view(-1,1)
        bands_input = torch.cat((
            b1_input,b2_input,b3_input,b4_input,b5_input,b6_input,b7_input),1).float()
        bands_input=bands_input.cuda()
        output=model(bands_input)
        loss = criterion(output, bands_input)
        MSE_loss = nn.MSELoss()(output, bands_input)
        # ===================backward====================
        optimizer.zero_grad()
#         MSE_loss.backward()
        loss.backward()
        optimizer.step()
        atuple.append(output)
        # ===================log========================
    if (epoch+1)%1==0:
        print('epoch [{}/{}], loss:{:.4f}, MSE_loss:{:.4f}'.format(epoch + 1, num_epochs, loss.data[0], MSE_loss.data[0]))
    
atuple=tuple(atuple)
#print(len(atuple))
inputdata = torch.cat(atuple,0)
#print(inputdata.size())

gtiff_driver = gdal.GetDriverByName('GTiff')
out_ds = gtiff_driver.Create(
    'out_seven_bands.tif',band1.size(1), band1.size(0), 7, in_band.DataType)
out_ds.SetProjection(in_ds.GetProjection())
out_ds.SetGeoTransform((386385.0+30*3000, 30.0, 0.0, 1873815.0-30.0*3751, 0.0, -30.0),)
for j in range(7):
    out_band = out_ds.GetRasterBand((7-j))
    readyData=(inputdata[:,j].view(band1.size(0),band1.size(1))).cpu().detach().numpy()
    out_band.WriteArray(readyData)
out_ds.FlushCache()
for k in range(1, 7):
    out_ds.GetRasterBand(k).ComputeStatistics(False)
del out_ds
readyData_blue=(inputdata[:,2].view(1,band1.size(0),band1.size(1))).cpu().detach().numpy()
readyData_green=(inputdata[:,3].view(1,band1.size(0),band1.size(1))).cpu().detach().numpy()
readyData_red=(inputdata[:,4].view(1,band1.size(0),band1.size(1))).cpu().detach().numpy()
readyData_BGR=torch.cat((readyData_blue,readyData_green,readyData_red),0)
save_image(readyData_BGR,'out_BRG.png')
print('all set!')

RuntimeError: invalid argument 2: size '[700 x 500]' is invalid for input with 102000 elements at ..\aten\src\TH\THStorage.cpp:84

In [16]:
import numpy as np
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image
import matplotlib.pyplot as plt
import os
import numpy
from PIL import Image
import torch
from osgeo import gdal


def min_max_normalization(tensor, min_value, max_value):
    min_tensor = tensor.min()
    tensor = (tensor - min_tensor)
    max_tensor = tensor.max()
    tensor = tensor / max_tensor
    tensor = tensor * (max_value - min_value) + min_value
    return tensor

class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(7, 3),
            nn.ReLU(True)
        )
        self.decoder = nn.Sequential(
            nn.Linear(3, 7),
            nn.ReLU(True)
            #nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x




os.chdir('E:/pyScripts/pytorch/data/142_49/LC08_L1TP_142049_20170120_20170311_01_T1/')

band_fn='LC08_L1TP_142049_20170120_20170311_01_T1_B1.TIF'
in_ds = gdal.Open(band_fn)
in_band=in_ds.GetRasterBand(1)
band1=torch.from_numpy(in_band.ReadAsArray().astype('float'))[4000:4700,3000:3500]
band1=min_max_normalization(band1,0.0,1.0)


band_fn='LC08_L1TP_142049_20170120_20170311_01_T1_B2.TIF'
in_ds = gdal.Open(band_fn)
in_band=in_ds.GetRasterBand(1)
band2=torch.from_numpy(in_band.ReadAsArray().astype('float'))[4000:4700,3000:3500]
band2=min_max_normalization(band2,0.0,1.0)


band_fn='LC08_L1TP_142049_20170120_20170311_01_T1_B3.TIF'
in_ds = gdal.Open(band_fn)
in_band=in_ds.GetRasterBand(1)
band3=torch.from_numpy(in_band.ReadAsArray().astype('float'))[4000:4700,3000:3500]
band3=min_max_normalization(band3,0.0,1.0)


band_fn='LC08_L1TP_142049_20170120_20170311_01_T1_B4.TIF'
in_ds = gdal.Open(band_fn)
in_band=in_ds.GetRasterBand(1)
band4=torch.from_numpy(in_band.ReadAsArray().astype('float'))[4000:4700,3000:3500]
band4=min_max_normalization(band4,0.0,1.0)


band_fn='LC08_L1TP_142049_20170120_20170311_01_T1_B5.TIF'
in_ds = gdal.Open(band_fn)
in_band=in_ds.GetRasterBand(1)
band5=torch.from_numpy(in_band.ReadAsArray().astype('float'))[4000:4700,3000:3500]
band5=min_max_normalization(band5,0.0,1.0)


band_fn='LC08_L1TP_142049_20170120_20170311_01_T1_B6.TIF'
in_ds = gdal.Open(band_fn)
in_band=in_ds.GetRasterBand(1)
band6=torch.from_numpy(in_band.ReadAsArray().astype('float'))[4000:4700,3000:3500]
band6=min_max_normalization(band6,0.0,1.0)


band_fn='LC08_L1TP_142049_20170120_20170311_01_T1_B7.TIF'
in_ds = gdal.Open(band_fn)
in_band=in_ds.GetRasterBand(1)
band7=torch.from_numpy(in_band.ReadAsArray().astype('float'))[4000:4700,3000:3500]
band7=min_max_normalization(band7,0.0,1.0)


num_epochs=300

learning_rate = 1e-4
#batch_size=55

model = autoencoder().cuda()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(
    model.parameters(), lr=learning_rate, weight_decay=1e-5)



for epoch in range(num_epochs):
    atuple=[]
    for i in range(0,band1.size(0)):
        b1_input=band1[i].view(-1,1)
        b2_input=band2[i].view(-1,1)
        b3_input=band3[i].view(-1,1)
        b4_input=band4[i].view(-1,1)
        b5_input=band5[i].view(-1,1)
        b6_input=band6[i].view(-1,1)
        b7_input=band7[i].view(-1,1)
        bands_input = torch.cat((
            b1_input,b2_input,b3_input,b4_input,b5_input,b6_input,b7_input),1).float()
        bands_input=bands_input.cuda()
        output=model(bands_input)
        loss = criterion(output, bands_input)
        MSE_loss = nn.MSELoss()(output, bands_input)
        # ===================backward====================
        optimizer.zero_grad()
#         MSE_loss.backward()
        loss.backward()
        optimizer.step()
        atuple.append(output)
        # ===================log========================
    if (epoch+1)%1==0:
        print('epoch [{}/{}], loss:{:.4f}, MSE_loss:{:.7f}'.format(epoch + 1, num_epochs, loss.data[0], MSE_loss.data[0]))
    
atuple=tuple(atuple)
#print(len(atuple))
inputdata = torch.cat(atuple,0)
#print(inputdata.size())

gtiff_driver = gdal.GetDriverByName('GTiff')
out_ds = gtiff_driver.Create(
    'out_seven_bands_times10000.tif',band1.size(1), band1.size(0), 7, in_band.DataType)
out_ds.SetProjection(in_ds.GetProjection())
out_ds.SetGeoTransform((386385.0+30*3000, 30.0, 0.0, 1873815.0-30.0*4000, 0.0, -30.0),)
#out_ds.SetGeoTransform(in_ds.GetGeoTransform())
for j in range(7):
    out_band = out_ds.GetRasterBand((j+1))
    readyData=(inputdata[:,j].view(band1.size(0),band1.size(1))).cpu().detach().numpy()
    readyData=readyData*10000
    out_band.WriteArray(readyData)
out_ds.FlushCache()
for k in range(1, 7):
    out_ds.GetRasterBand(k).ComputeStatistics(False)
del out_ds
readyData_blue=(inputdata[:,2].view(1,band1.size(0),band1.size(1)))
readyData_green=(inputdata[:,3].view(1,band1.size(0),band1.size(1)))
readyData_red=(inputdata[:,4].view(1,band1.size(0),band1.size(1)))
readyData_BGR=torch.cat((readyData_blue,readyData_green,readyData_red),0)
save_image(readyData_BGR,'out_BRG.png')
print('all set!')

E:\Installations\Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:132: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


epoch [1/300], loss:2.7804, MSE_loss:0.0474
epoch [2/300], loss:2.7590, MSE_loss:0.0408
epoch [3/300], loss:2.7455, MSE_loss:0.0368
epoch [4/300], loss:2.7348, MSE_loss:0.0341
epoch [5/300], loss:2.7262, MSE_loss:0.0323
epoch [6/300], loss:2.7212, MSE_loss:0.0313
epoch [7/300], loss:2.7193, MSE_loss:0.0309
epoch [8/300], loss:2.7189, MSE_loss:0.0307
epoch [9/300], loss:2.7187, MSE_loss:0.0306
epoch [10/300], loss:2.7185, MSE_loss:0.0306
epoch [11/300], loss:2.7183, MSE_loss:0.0305
epoch [12/300], loss:2.7183, MSE_loss:0.0304
epoch [13/300], loss:2.7185, MSE_loss:0.0304
epoch [14/300], loss:2.7183, MSE_loss:0.0304
epoch [15/300], loss:2.7185, MSE_loss:0.0304
epoch [16/300], loss:2.7185, MSE_loss:0.0304
epoch [17/300], loss:2.7183, MSE_loss:0.0304
epoch [18/300], loss:2.7184, MSE_loss:0.0304
epoch [19/300], loss:1.9267, MSE_loss:0.0295
epoch [20/300], loss:1.9047, MSE_loss:0.0288
epoch [21/300], loss:0.4840, MSE_loss:0.0187
epoch [22/300], loss:0.4515, MSE_loss:0.0134
epoch [23/300], los

epoch [182/300], loss:0.4019, MSE_loss:0.0001
epoch [183/300], loss:0.4019, MSE_loss:0.0001
epoch [184/300], loss:0.4019, MSE_loss:0.0001
epoch [185/300], loss:0.4019, MSE_loss:0.0001
epoch [186/300], loss:0.4019, MSE_loss:0.0001
epoch [187/300], loss:0.4018, MSE_loss:0.0001
epoch [188/300], loss:0.4019, MSE_loss:0.0001
epoch [189/300], loss:0.4018, MSE_loss:0.0001
epoch [190/300], loss:0.4018, MSE_loss:0.0001
epoch [191/300], loss:0.4018, MSE_loss:0.0001
epoch [192/300], loss:0.4018, MSE_loss:0.0001
epoch [193/300], loss:0.4019, MSE_loss:0.0001
epoch [194/300], loss:0.4019, MSE_loss:0.0001
epoch [195/300], loss:0.4018, MSE_loss:0.0001
epoch [196/300], loss:0.4021, MSE_loss:0.0002
epoch [197/300], loss:0.4020, MSE_loss:0.0002
epoch [198/300], loss:0.4019, MSE_loss:0.0002
epoch [199/300], loss:0.4019, MSE_loss:0.0002
epoch [200/300], loss:0.4019, MSE_loss:0.0001
epoch [201/300], loss:0.4018, MSE_loss:0.0001
epoch [202/300], loss:0.4018, MSE_loss:0.0001
epoch [203/300], loss:0.4018, MSE_

In [17]:
hiddenlayer_tupple=[]
for i in range(0,band1.size(0)):
    b1_input=band1[i].view(-1,1)
    b2_input=band2[i].view(-1,1)
    b3_input=band3[i].view(-1,1)
    b4_input=band4[i].view(-1,1)
    b5_input=band5[i].view(-1,1)
    b6_input=band6[i].view(-1,1)
    b7_input=band7[i].view(-1,1)
    bands_input = torch.cat((
        b1_input,b2_input,b3_input,b4_input,b5_input,b6_input,b7_input),1).float()
    bands_input=bands_input.cuda()
    output=model.encoder(bands_input)
    hiddenlayer_tupple.append(output)
        
hiddenlayer_tupple=tuple(hiddenlayer_tupple)
#print(len(atuple))
hiddenlayer_data = torch.cat(hiddenlayer_tupple,0)
#print(inputdata.size())

gtiff_driver = gdal.GetDriverByName('GTiff')
out_ds = gtiff_driver.Create(
    'out_hiddenlayers_times10000.tif',band1.size(1), band1.size(0), 3, in_band.DataType)
out_ds.SetProjection(in_ds.GetProjection())
out_ds.SetGeoTransform((386385.0+30*3000, 30.0, 0.0, 1873815.0-30.0*4000, 0.0, -30.0),)
#out_ds.SetGeoTransform(in_ds.GetGeoTransform())
for j in range(3):
    out_band = out_ds.GetRasterBand((j+1))
    readyData=(hiddenlayer_data[:,j].view(band1.size(0),band1.size(1))).cpu().detach().numpy()
    readyData=readyData*10000
    print(readyData.shape)
    out_band.WriteArray(readyData)
out_ds.FlushCache()
for k in range(1, 3):
    out_ds.GetRasterBand(k).ComputeStatistics(False)
del out_ds
readyData_blue=(hiddenlayer_data[:,0].view(1,band1.size(0),band1.size(1)))
readyData_green=(hiddenlayer_data[:,1].view(1,band1.size(0),band1.size(1)))
readyData_red=(hiddenlayer_data[:,2].view(1,band1.size(0),band1.size(1)))
readyData_BGR=torch.cat((readyData_blue,readyData_green,readyData_red),0)
save_image(readyData_BGR,'out_hidden_BRG.png')
print('all set!')

(700, 500)
(700, 500)
(700, 500)
all set!


# Whole IMG

In [20]:
band_fn='LC08_L1TP_142049_20170120_20170311_01_T1_B1.TIF'
in_ds = gdal.Open(band_fn)
in_band=in_ds.GetRasterBand(1)
band1=torch.from_numpy(in_band.ReadAsArray().astype('float'))
band1=min_max_normalization(band1,0.0,1.0)


band_fn='LC08_L1TP_142049_20170120_20170311_01_T1_B2.TIF'
in_ds = gdal.Open(band_fn)
in_band=in_ds.GetRasterBand(1)
band2=torch.from_numpy(in_band.ReadAsArray().astype('float'))
band2=min_max_normalization(band2,0.0,1.0)


band_fn='LC08_L1TP_142049_20170120_20170311_01_T1_B3.TIF'
in_ds = gdal.Open(band_fn)
in_band=in_ds.GetRasterBand(1)
band3=torch.from_numpy(in_band.ReadAsArray().astype('float'))
band3=min_max_normalization(band3,0.0,1.0)


band_fn='LC08_L1TP_142049_20170120_20170311_01_T1_B4.TIF'
in_ds = gdal.Open(band_fn)
in_band=in_ds.GetRasterBand(1)
band4=torch.from_numpy(in_band.ReadAsArray().astype('float'))
band4=min_max_normalization(band4,0.0,1.0)


band_fn='LC08_L1TP_142049_20170120_20170311_01_T1_B5.TIF'
in_ds = gdal.Open(band_fn)
in_band=in_ds.GetRasterBand(1)
band5=torch.from_numpy(in_band.ReadAsArray().astype('float'))
band5=min_max_normalization(band5,0.0,1.0)


band_fn='LC08_L1TP_142049_20170120_20170311_01_T1_B6.TIF'
in_ds = gdal.Open(band_fn)
in_band=in_ds.GetRasterBand(1)
band6=torch.from_numpy(in_band.ReadAsArray().astype('float'))
band6=min_max_normalization(band6,0.0,1.0)


band_fn='LC08_L1TP_142049_20170120_20170311_01_T1_B7.TIF'
in_ds = gdal.Open(band_fn)
in_band=in_ds.GetRasterBand(1)
band7=torch.from_numpy(in_band.ReadAsArray().astype('float'))
band7=min_max_normalization(band7,0.0,1.0)


outlayers_tupple = []
hiddenlayer_tupple=[]
for i in range(0,band1.size(0)):
    b1_input=band1[i].view(-1,1)
    b2_input=band2[i].view(-1,1)
    b3_input=band3[i].view(-1,1)
    b4_input=band4[i].view(-1,1)
    b5_input=band5[i].view(-1,1)
    b6_input=band6[i].view(-1,1)
    b7_input=band7[i].view(-1,1)
    bands_input = torch.cat((
        b1_input,b2_input,b3_input,b4_input,b5_input,b6_input,b7_input),1).float()
    bands_input=bands_input.cuda()
    
    outputlayers=model(bands_input)
#     output_hidden=model.encoder(bands_input)
    
    outlayers_tupple.append(outputlayers)
#     hiddenlayer_tupple.append(output_hidden)

outlayers_tupple=tuple(outlayers_tupple)
# hiddenlayer_tupple=tuple(hiddenlayer_tupple)

outlayer_data = torch.cat(outlayers_tupple,0)
# hiddenlayer_data = torch.cat(hiddenlayer_tupple,0)


gtiff_driver = gdal.GetDriverByName('GTiff')
out_ds = gtiff_driver.Create(
    'out_wholeIMG_times10000.tif',band1.size(1), band1.size(0), 7, in_band.DataType)
out_ds.SetProjection(in_ds.GetProjection())
out_ds.SetGeoTransform(in_ds.GetGeoTransform())
for j in range(7):
    out_band = out_ds.GetRasterBand((j+1))
    readyData=(outlayer_data[:,j].view(band1.size(0),band1.size(1))).cpu().detach().numpy()
    readyData=readyData*10000
    print(readyData.shape)
    out_band.WriteArray(readyData)
out_ds.FlushCache()
for k in range(1, 7):
    out_ds.GetRasterBand(k).ComputeStatistics(False)
del out_ds
readyData_blue=(outlayer_data[:,1].view(1,band1.size(0),band1.size(1)))
readyData_green=(outlayer_data[:,2].view(1,band1.size(0),band1.size(1)))
readyData_red=(outlayer_data[:,3].view(1,band1.size(0),band1.size(1)))
readyData_BGR=torch.cat((readyData_blue,readyData_green,readyData_red),0)
save_image(readyData_BGR,'out_wholeIMG_BRG.png')



# gtiff_driver = gdal.GetDriverByName('GTiff')
# out_ds = gtiff_driver.Create(
#     'out_hiddenwholeIMG_times10000.tif',band1.size(1), band1.size(0), 3, in_band.DataType)
# out_ds.SetProjection(in_ds.GetProjection())
# out_ds.SetGeoTransform((386385.0+30*3000, 30.0, 0.0, 1873815.0-30.0*4000, 0.0, -30.0),)
# #out_ds.SetGeoTransform(in_ds.GetGeoTransform())
# for j in range(3):
#     out_band = out_ds.GetRasterBand((j+1))
#     readyData=(hiddenlayer_data[:,j].view(band1.size(0),band1.size(1))).cpu().detach().numpy()
#     readyData=readyData*10000
#     print(readyData.shape)
#     out_band.WriteArray(readyData)
# out_ds.FlushCache()
# for k in range(1, 3):
#     out_ds.GetRasterBand(k).ComputeStatistics(False)
# del out_ds
# readyData_blue=(hiddenlayer_data[:,0].view(1,band1.size(0),band1.size(1)))
# readyData_green=(hiddenlayer_data[:,1].view(1,band1.size(0),band1.size(1)))
# readyData_red=(hiddenlayer_data[:,2].view(1,band1.size(0),band1.size(1)))
# readyData_BGR=torch.cat((readyData_blue,readyData_green,readyData_red),0)
# save_image(readyData_BGR,'out_hiddenwholeIMG_BRG.png')



print('all set!')

MemoryError: 

In [20]:
import numpy as np
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image
import matplotlib.pyplot as plt
import os
import numpy
from PIL import Image
import torch
from osgeo import gdal

def min_max_normalization(tensor, min_value, max_value):
    min_tensor = tensor.min()
    tensor = (tensor - min_tensor)
    max_tensor = tensor.max()
    tensor = tensor / max_tensor
    tensor = tensor * (max_value - min_value) + min_value
    return tensor

os.chdir('E:/pyScripts/pytorch/data/142_49/LC08_L1TP_142049_20170120_20170311_01_T1/')
raw=[]
for i in range(7):
    band_fn='windowed_LC08_L1TP_142049_20170120_20170311_01_T1_B'+str(i+1)+'.rst'
    in_ds = gdal.Open(band_fn)
    in_band=in_ds.GetRasterBand(1)
    band=torch.from_numpy(in_band.ReadAsArray().astype('float'))
    band=min_max_normalization(band,0.0,1.0)
    raw.append(band)

pca_inverse=[]
for i in range(7):
    band_fn='inverse_pca'+str(i+1)+'.rst'
    in_ds = gdal.Open(band_fn)
    in_band=in_ds.GetRasterBand(1)
    band=torch.from_numpy(in_band.ReadAsArray().astype('float'))
    band=min_max_normalization(band,0.0,1.0)
    pca_inverse.append(band)
    
ae_reconstructed=[]
for i in range(7):
    band_fn='AE_outputs_whole_b'+str(i+1)+'.rst'
    in_ds = gdal.Open(band_fn)
    in_band=in_ds.GetRasterBand(1)
    band=torch.from_numpy(in_band.ReadAsArray().astype('float'))
    band=min_max_normalization(band,0.0,1.0)
    ae_reconstructed.append(band)

MSE_loss_pca = []
BCE_loss_pca = []
for i in range(7):
    loss=nn.MSELoss()(pca_inverse[0], raw[0])
    bceloss=nn.BCELoss()(pca_inverse[0], raw[0])
    MSE_loss_pca.append(loss)
    BCE_loss_pca.append(bceloss)
    
MSE_loss_ae = []
BCE_loss_ae = []
for i in range(7):
    loss=nn.MSELoss(size_average=False)(ae_reconstructed[0], raw[0])
    bceloss=nn.BCELoss()(ae_reconstructed[0], raw[0])
    MSE_loss_ae.append(loss)
    BCE_loss_ae.append(bceloss)

# for i in range(7):
#     print('MSE Loss of AE: ', MSE_loss_ae[i], 'MSE Loss of PCA: ', MSE_loss_pca[i])
#     print('Cross Entropy Loss of AE: ', BCE_loss_ae[i], 'Cross Entropy Loss of PCA: ', BCE_loss_pca[i])


tensor(1., dtype=torch.float64)


In [38]:
    
# for i in range(7):
#     print(nn.BCELoss(size_average=False)(pca_inverse[i], raw[i]))
    

tensor(25583.9812, dtype=torch.float64)
tensor(9314.7646, dtype=torch.float64)
tensor(2714.9704, dtype=torch.float64)
tensor(27777.6520, dtype=torch.float64)
tensor(1012.9957, dtype=torch.float64)
tensor(880.2447, dtype=torch.float64)
tensor(1292.6850, dtype=torch.float64)


E:\Installations\Visual Studio\Shared\Anaconda3_64\lib\site-packages\torch\nn\functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [39]:
print(raw[0].size(), pca_inverse[0].size(), ae_reconstructed[0].size())

torch.Size([4000, 4000]) torch.Size([4000, 4000]) torch.Size([4000, 4000])


In [46]:
from statsmodels.formula.api import ols
import pandas

for i in range(7):
    x = ae_reconstructed[i].view(-1).numpy()
    y= raw[i].view(-1).detach().numpy()
    data = pandas.DataFrame({'x': x, 'y': y})
    model = ols("y ~ x", data).fit()
    print(model.summary()) 

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.942
Model:                            OLS   Adj. R-squared:                  0.942
Method:                 Least Squares   F-statistic:                 2.613e+08
Date:                Wed, 31 Oct 2018   Prob (F-statistic):               0.00
Time:                        17:03:26   Log-Likelihood:             4.1506e+07
No. Observations:            16000000   AIC:                        -8.301e+07
Df Residuals:                15999998   BIC:                        -8.301e+07
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0454    8.9e-06  -5094.738      0.0

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.995
Model:                            OLS   Adj. R-squared:                  0.995
Method:                 Least Squares   F-statistic:                 3.239e+09
Date:                Wed, 31 Oct 2018   Prob (F-statistic):               0.00
Time:                        17:03:54   Log-Likelihood:             6.8050e+07
No. Observations:            16000000   AIC:                        -1.361e+08
Df Residuals:                15999998   BIC:                        -1.361e+08
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0055   1.19e-06   4585.580      0.0